<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ageestimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 重回帰分析の応用: 顔画像からの年齡推定



## はじめに

重回帰分析の応用例として，顔画像からそこに写った人物の年齡を推定するデモを作ってみました．
推定精度は高くありませんが，こんなところでも多変量解析手法が使えるよ，ということが伝わればいいな．
ちなみに，推定精度を高めたければ，もっと複雑なモデルを使うことになります．
「機械学習I,II」で学べるかも．

人の顔が中央に大きく写った幅64画素高さ64画素の3チャンネルカラー画像 38,138 枚があります．
これらの画像に写った人物の年齡を表す整数値も与えられています．
このとき，一つの画像の画素値を1列にならべて $64\times 64\times 3 = 12288$ 次元ベクトルとみなせば，そこから年齡を予測する問題を重回帰分析の問題として定式化できます．

ひとつの顔に対応する $12288$ 次元ベクトルに $1$ を付け足した $12289$ 次元ベクトルを $\mathbf{x}$ とおき，そのひとの年齡を $y$ とおくとき，

$$
\mathbf{y} \approx \mathbf{w}^{\top}\cdot\mathbf{x}
$$

となるような $(D+1)$次元ベクトル $\mathbf{w}$ を最小二乗法によって求める，というわけです（実際には，データの次元数が大きすぎるので，主成分分析を利用した次元削減も併用しています．詳しくはこの notebook の後の方に書いてます）．

この notebook では，上記の設定で既に学習済みの $\mathbf{w}$ を用いて，適当な画像 $\mathbf{x}$ を与えて年齡推定をさせてみます．


## 準備

In [ ]:
# 必要なパッケージのインポート
import numpy as np
import cv2  # Python による「コンピュータビジョン(Computer Vision)」のためのライブラリ OpenCV のパッケージをインポート

# 顔検出器の準備
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

次の3つのセルではあとで使う関数を定義しています．これらのセルを実行しただけでは何も起こりませんが，あらかじめ実行して関数を定義しておかないと後のセルが動きません．

In [ ]:
# Colab へのファイルアップロードを実行する関数
#
def uploadToColab():
    try:
        from google.colab import files
        rv = files.upload()
    except:
        print('このコードは Colab 以外の環境では実行できないよ．')


In [ ]:
# OpenCVの形式の画像を表示する関数
#
def imshow(img):
    try:
        from google.colab.patches import cv2_imshow
        cv2_imshow(img) # Colab上で実行している場合
    except:
        cv2.imshow(img)  # それ以外の場合

In [ ]:
# 与えられた画像の中から顔を検出する
#
def faceDetector(img, faceCascade, maxSize=400):

    h, w = img.shape[:2]

    # (短辺の長さ) <= maxSize にリサイズ
    if min(w, h) > maxSize:
        if w <= h:
            w2, h2 = maxSize, h*maxSize//w
        else:
            w2, h2 = w*maxSize//h, maxSize
        imgDisp = cv2.resize(img, (w2, h2))
    else:
        imgDisp = np.copy(img)

    # 顔検出を実行
    imgGray = cv2.cvtColor(imgDisp, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(imgGray, 1.1, 4)
    nFace = len(faces)

    # 検出した顔のうち最初のひとりを選んで切り取り
    if nFace > 0:
        print(faces)
        x, y, ww, hh = faces[0]
        imgFace = np.copy(imgDisp[y:y+hh, x:x+ww, :])
        cv2.rectangle(imgDisp, (x, y), (x+ww, y+hh), color=(0, 255, 0), thickness=2)
        return nFace, imgDisp, imgFace
    else:
        return nFace, imgDisp, None


$\mathbf{w}$ などのパラメータを読み込みます．

In [ ]:
# takataka のウェブサイトからパラメータを格納したファイルを Colab へダウンロード
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/ageestimation.npz

import os
path = 'ageestimation.npz'
if os.path.exists(path):
    # パラメータを読み込む
    params = np.load(path)
    print(f'ファイル {path} を読み込みました')
    Xm  = params['Xm']  # 平均ベクトル
    eve = params['eve'] # 主成分分析で得られた固有ベクトル
    w   = params['w']   # 重回帰分析で得られたパラメータ
    print(Xm.shape, eve.shape, w.shape)
else:
    print(f'ファイル {path} の読み込みに失敗したようです．再実行してみてください')

## 実験

(1) 画像を Colab へアップロードします．次のような画像にしてください

- ひとりのひとのほぼ正面を向いた顔全体が写ってる．複数人を検出した場合，ひとりだけ抜き出します（あとで条件に合わない画像をわざと与えて実験してみるのもよいでしょう）．
- 画像全体に顔がドアップで写っているような場合はうまく顔検出ができないかもしれません．顔がもう少し小さく写ってる画像を探してみましょう．
- ファイル名に空白やマルチバイト文字（日本語など）が含まれているとうまく動作しない場合があるので，自分のPCの方で適当な名前に変えておく（拡張子は変えてはいけない）．


In [ ]:
# Colab へファイルをアップロード
uploadToColab()

# ls コマンドでファイルを一覧
! ls

上のセルを実行してファイルをアップロードできたら，次のセルのファイル名の部分にその名前を指定して，読み込ませましょう．

In [ ]:
# 画像を読み込む．`hoge.jpg` を自分がアップロードしたファイルの名前に修正
img = cv2.imread('hoge.jpg')

(2) 顔検出を実行して顔領域を切り取った画像を作る

In [ ]:
numFace, imgDisp, imgFace = faceDetector(img, faceCascade, maxSize=400)
if numFace > 0:
    imshow(imgDisp)
    imshow(imgFace)
else:
    imshow(imgDisp)
    print('顔を検出できませんでした')

(3) 顔を所定の大きさにリサイズしてから次元削減し，年齡推定を実行．

In [ ]:
# 顔画像を 64 x 64 にリサイズして 12288次元ベクトルに
xvec = cv2.resize(imgFace, (64, 64)).reshape(-1)
print(xvec.shape)

# PCAを利用した次元削減
yvec = (xvec - Xm) @ U
print(yvec.shape)

# 予測値の計算
yvec1 = np.concatenate(([1], yvec)) # 先頭に 1 をくっつける
z = w @ yvec1

# 結果の表示
print(f'このひとの年齡の推定値は {z:.1f} です')

## 解説

この年齡推定の仕組みは，次のようにして作りました．

(1) [IMDB-WIKI dataset](https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/) という研究用画像データセットの中に，Wikipedia から集めた顔画像とその性別，生年月日，撮影年などが含まれたデータ（画像 62,328枚）があったので，これを利用．

(2) 生年月日と撮影年から年齡を算出．顔がちゃんと写ってなかったり年齡の推定値がおかしかったりするデータを削除．顔の領域のみを抜き出す．得られた画像は 38138 枚．この前処理の部分では，こちらのサイトを参考にさせてもらいました：
[顔画像から年齢・性別を推定するためのデータセットIMDB-WIKI](https://qiita.com/yu4u/items/a2410f46669c5f20ee8e)

(3) 得られた画像を $64\times 64$ に縮小してデータ行列 `X` を作成．
`X.shape` は `(38138, 12288)`．

(4) `X` の平均を求めてこれを `X` から引き，特異値分解経由で主成分分析のための固有値・固有ベクトルを算出．次のスペックの Mac で `np.linalg.svd` の実行に23分．
- CPU: Apple M1 Pro 
- メモリ: 32GB

(5) 累積寄与率を調べると，2151次元ではじめて99%を超えていたので，2151次元に次元削減した．

(6) そのデータを用いて最小二乗法の解を計算．上記の Mac で `np.linalg.lsesq` の実行に 28 秒．

(7) `X` の平均（12288次元），固有ベクトル（12288x2151），重回帰のパラメータ（2152次元）をファイルに保存．